In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/niel77/MechanicalAgents/data/Examples_small.pdf")
pages = loader.load_and_split()

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(pages)

In [25]:

from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [26]:
from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="nomic-embed-text",
# )


In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
persist_directory = "./Examples_db"

# Create the directory if it doesn't exist
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)
db = Chroma.from_documents(chunks, embeddings_model, persist_directory=persist_directory)

In [28]:
CHROMA_DB_PATH="/home/niel77/MechanicalAgents/chromadb"
CHROMA_COLLECTION="autogen-docs-test"

import chromadb

chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = chroma_client.get_or_create_collection(name=CHROMA_COLLECTION)


vector_db = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function = embeddings)

In [36]:
from autogen import ConversableAgent
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent



config_list = [
    {
        'base_url': 'http://localhost:11434/',
        'api_key' : 'ollama',
        'model': 'phi3:mini'
    }
]

llm_config = {
    "seed": 25,
    "temperature": 0,
    "config_list": config_list,
    }


In [37]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
import chromadb

assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config=False,
    retrieve_config={
        "model": config_list[0]["model"],
        "task": "default",
        "update_context": True,
        "n_results": 3,
        "docs_path":[
            "/home/niel77/MechanicalAgents/data/Examples_small.md",
            ],
       "get_or_create": True,
       "overwrite": False,
       "vector_db": vector_db,
       "collection_name": CHROMA_COLLECTION,
    },
)


In [38]:
assistant.reset()
query = "Write CadQuery code to create a plate with hole."
ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=query)

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2
Model phi3:mini not found. Using cl100k_base encoding.


Trying to create collection.
doc_ids:  [['doc_0', 'doc_1']]
Adding content of doc doc_0 to context.


Model phi3:mini not found. Using cl100k_base encoding.


Adding content of doc doc_1 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

If user's intent is question answering, you must give as short an answer as possible.

User's question is: Write CadQuery code to create a plate with hole.

Context is: ﻿**CadQuery 

ChatResult(chat_id=None, chat_history=[{'content': 'You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user. You should follow the following steps to answer a question:\nStep 1, you estimate the user\'s intent based on the question and context. The intent can be a code generation task or\na question answering task.\nStep 2, you reply based on the intent.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nIf user\'s intent is code generation, you must obey the following rules:\nRule 1. You MUST NOT install any packages because all the packages needed are already installed.\nRule 2. You must follow the formats below to write your code:\n```language\n# your code\n```\n\nIf user\'s intent is question answering, you must give as short an answer as possible.\n\nUser\'s question is: Write CadQuery code to create a plate with hole.\n\nContext is: \ufeff*